In [1]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 114.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 117.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.2 MB/s eta 0:00:00


In [2]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 9.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from simpletransformers.classification import ClassificationModel
import numpy as np
import contractions
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [34]:
df = pd.read_csv('/content/drive/MyDrive/NLP/combined_file.csv')

In [35]:
df = df.dropna(subset=['text'])
df = df.reset_index(drop=True)
df['text'] = df['text'].str.lower()
url_pattern = r'http[s]?://[^\s]+'
urls_found = df[df['text'].str.contains(url_pattern, regex=True)]
df['text'] = df['text'].str.replace(url_pattern, '', regex=True)

email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
emails_found = df[df['text'].str.contains(email_pattern, regex=True)]
df['text'] = df['text'].str.replace(email_pattern, '', regex=True)

hashtag_pattern = r'#[\w]+'
hashtags_found = df[df['text'].str.contains(hashtag_pattern, regex=True)]
df['text'] = df['text'].str.replace(hashtag_pattern, '', regex=True)

df['text'] = df['text'].str.strip().str.replace('\s+', ' ', regex=True)
df['text'] = df['text'].apply(contractions.fix)

def remove_special_characters(text):
    return re.sub(r'[^A-Za-z0-9\s]', '', text)

df['text'] = df['text'].apply(remove_special_characters)

def remove_numbers(text):
    return re.sub(r'\d+', '', text)

df['text'] = df['text'].apply(remove_numbers)

df['text'] = df['text'].str.strip().str.replace('\s+', ' ', regex=True)

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def preprocess(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    pos_tags = pos_tag(filtered_tokens)
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]
    return ' '.join(lemmatized_tokens)

df['text'] = df['text'].apply(preprocess)

In [36]:
data = df[['text', 'score']]

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92670 entries, 0 to 92669
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    92670 non-null  object
 1   score   92659 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB


In [38]:
data = data.dropna(subset=['score'])

In [39]:
non_numeric_scores = data[pd.to_numeric(data['score'], errors='coerce').isna()]
non_numeric_scores

,text,score
59434,reject peel commission idea partition palestin...,000 for five years and ruled that further immi...
59435,would function provisional governmentinexile a...,they were concerned over the heavy losses the...
59437,young jew sail palestine take mandatory palest...,initially under a different name. Lehi was a ...
59440,effectively end left hitherto unrivalled domin...,promoting a socially conservative and economi...
72394,etc propalestinians truly care much life gaza,why are they not calling for the surrender of...


In [40]:
data = data[pd.to_numeric(data['score'], errors='coerce').notna()]

In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 92654 entries, 0 to 92669
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    92654 non-null  object
 1   score   92654 non-null  object
dtypes: object(2)
memory usage: 2.1+ MB


In [42]:
data = data.reset_index(drop=True)

In [43]:
data['score'] = pd.to_numeric(data['score'])

In [44]:
data['text'] = data['text'].astype(str)

In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92654 entries, 0 to 92653
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    92654 non-null  object 
 1   score   92654 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.4+ MB


In [46]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [47]:
kf = KFold(n_splits=10, shuffle=True, random_state=5805)
mse_scores = []

In [48]:
def train_model(fold, train_df, test_df):
    print(f"\nStarting Fold {fold + 1}...")
    model = ClassificationModel(
        "bert",
        "bert-base-uncased",
        num_labels=1,
        args={
            "save_model_every_epoch": False,
            "overwrite_output_dir": True,
            "regression": True,
            "learning_rate": 2e-5,
            "train_batch_size": 16,
            "eval_batch_size": 16,
            "max_seq_length": 512,
            "gradient_accumulation_steps": 2,
            "fp16": False,
            "use_multiprocessing": False,
            "use_multiprocessing_for_evaluation": False,
            "process_count": 0,
            "dataloader_num_workers": 0,
            "no_cache": True,
            "logging_level": "debug",
        },
    )

    for p in model.model.bert.embeddings.parameters():
        p.requires_grad = False
    for layer in model.model.bert.encoder.layer[:8]:
        for p in layer.parameters():
            p.requires_grad = False


    model.train_model(train_df)
    predictions, _ = model.predict(list(test_df['text']))
    mse = mean_squared_error(test_df['score'], predictions)
    print(f"MSE on Fold {fold + 1}: {mse}")

    return mse

In [52]:
data = data[:45000]

In [54]:
data

,text,score
0,message live verify many scammer try fool,1.0
1,salaam everyone f currently go divorce making ...,1.0
2,quran saw verse case say happy come sad go cal...,1.0
3,holocaust book family jewish hungarian dwarf s...,1.0
4,shalom friend baal teshuva many tattoo arm usu...,1.0
...,...,...
44995,try defend people say hamas terrorist war exac...,48.0
44996,,1.0
44997,,1.0
44998,bad quality sorry could find good,1.0


In [55]:
for fold, (train_index, test_index) in enumerate(kf.split(data)):
    train_df = data.iloc[train_index].copy()
    test_df = data.iloc[test_index].copy()
    mse = train_model(fold, train_df, test_df)
    mse_scores.append(mse)


Starting Fold 1...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/2532 [00:00<?, ?it/s]

  0%|          | 0/282 [00:00<?, ?it/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/2532 [00:00<?, ?it/s]

  0%|          | 0/282 [00:00<?, ?it/s]

MSE on Fold 10: 294.28328589153006


In [56]:
average_mse = np.mean(mse_scores)
print(f"\nAverage MSE across {kf.n_splits} folds: {average_mse}")
print("MSE Scores:", mse_scores)


Average MSE across 10 folds: 532.1835117871276
MSE Scores: [115.33005103033778, 380.6635658120066, 675.4687039945294, 246.68230745723713, 587.5802091136591, 248.96024680079054, 149.8327168485024, 619.8104285089831, 2003.2236024136998, 294.28328589153006]
